In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:



def prepare_state(theta, phi):
    """Quantum function to prepare states of the form
        cos(\theta/2) |0> + e^(i\phi) sin(\theta/2) |1>.

    In principle, this function would be hidden, with no knowledge of theta/phi,
    in order to prepare a truly random state. It is provided here just for
    expository purposes.
    """
    qml.RY(theta, wires=0)
    qml.RZ(phi, wires=0)

    return qml.state()


def extract_bloch_vector(theta, phi):
    """Given a set of angular parameters representing the quantum state
        cos(\theta/2) |0> + e^(i\phi) sin(\theta/2) |1>,
    compute the Bloch vector associated to this state.

    The Bloch vector has three real-valued elements representing the position of
    the quantum state in the 3-dimensional space of the Bloch sphere. It can
    be computed by measuring a cleverly-chosen set of expectation values.

    Args:
        theta (float): Angular parameter of the states.
        phi (float): Phase parameter of the state.

    Returns:
        bloch_vector (array[float, float, float]): A NumPy array representing the
        3-element Bloch vector.
    """
    # Note here that we use an analytic device so that we get exact results
    dev = qml.device("default.qubit", wires=1)

    # YOUR CODE HERE
    # Use theta, phi *only* as arguments to prepare_state

    x = np.cos(phi) * np.sin(theta)
    y = np.sin(phi) * np.sin(theta)
    z = np.cos(theta)

    return np.array([x, y, z])

In [34]:
def part_a():
    """Write a quantum circuit that computes the Boolean function
        f(a, b, c, d) = ab + cd
    where the + here represents the XOR operation.

    The first four qubits of your circuit should correspond to the
    input variables, a, b, c, d, in that order. The last qubit of your
    circuit should correspond to the output. If you use an auxiliary wires
    in your circuit, these should be uncomputed, like so:
          _______
     a ---|  C  |--- a
     b ---|  I  |--- b
     c ---|  R  |--- c
     d ---|  C  |--- d
     0 ---|     |--- 0 # Optional aux. wires
     0 ---|     |--- 0
          ...
     0 ---=======--- ab + cd

    """

    # You can change this to as many wires as you need
    num_wires = 7

    # Do not change the number of shots on the device
    dev = qml.device("default.qubit", wires=num_wires, shots=1)

    @qml.qnode(dev)
    def my_circuit(a, b, c, d):
        """ Args:
                a, b, c, d (int): The input variables, either 0 or 1.

            Returns:
                array[int]: A single sample of the output of your circuit.
        """

        # YOUR CODE HERE
        # Apply some quantum gates
        # Don't forget to initialize the first 4 wires based on a, b, c, d

        if a == 1:
            qml.PauliX(wires=0)
        if b == 1:
            qml.PauliX(wires=1)
        if c == 1:
            qml.PauliX(wires=3)
        if d == 1:
            qml.PauliX(wires=4)


        qml.Toffoli(wires=[0,1,2])
        qml.Toffoli(wires=[3,4,5])
        qml.CNOT(wires=[2, 5])

        # wires 2, 6 not used

        return qml.sample(wires=[0,1,3,4,2,6,5])

    return my_circuit

In [38]:
part_a()(1,1,1,1)

tensor([1, 1, 1, 1, 1, 0, 0], requires_grad=True)

In [ ]:
def part_b():
    """Write a quantum circuit that computes the Boolean function
        f(a, b, c) = NOT(a) * b * c
    where the * here represents the AND operation.

    The first three qubits of your circuit should correspond to the
    input variables, a, b, c, in that order. The last qubit of your
    circuit should correspond to the output. If you use an auxiliary wires
    in your circuit, these should be uncomputed.

          _______
     a ---|  C  |--- a
     b ---|  I  |--- b
     c ---|  R  |--- c
     0 ---|  C  |--- 0 # Optional aux. wires
     0 ---|     |--- 0
            ...
     0 ---=======--- NOT(a) * b * c

    """

    # You can change this to as many wires as you need
    num_wires = 7

    # Do not change the number of shots on the device
    dev = qml.device("default.qubit", wires=num_wires, shots=1)

    @qml.qnode(dev)
    def my_circuit(a, b, c):
        """ Args:
                a, b, c (int): The input variables, either 0 or 1.

            Returns:
                array[int]: A single sample of the output of your circuit.
        """

        # YOUR CODE HERE
        # Apply some quantum gates
        if a == 1:
            qml.PauliX(wires=0)
        if b == 1:
            qml.PauliX(wires=1)
        if c == 1:
            qml.PauliX(wires=2)

        # C of CNOT
        qml.PauliX(wires=4)
        qml.CNOT(wires=[4, 0])

        #


        qml.Toffoli(wires=[0, 1, ])
        qml.Toffoli()


        return qml.sample(wires=[0,1,2,3,4,5,6])

    return my_circuit

In [ ]:

def part_c():
    """Write a quantum circuit that computes the Boolean function
        f(a, b, c) = a + a * NOT(b) + NOT(a * b * c)
    where the * here represents the AND operation, and + the XOR.

    The first three qubits of your circuit should correspond to the
    input variables, a, b, c, in that order. The last qubit of your
    circuit should correspond to the output. If you use an auxiliary wires
    in your circuit, these should be uncomputed.
          _______
     a ---|  C  |--- a
     b ---|  I  |--- b
     c ---|  R  |--- c
     0 ---|  C  |--- 0 # Optional aux. wires
     0 ---|     | 0
            ...
     0 ---=======---- a + a * NOT(b) + NOT(a * b * c)

    """

    # You can change this to as many wires as you need
    num_wires = 5

    # Do not change the number of shots on the device
    dev = qml.device("default.qubit", wires=num_wires, shots=1)

    @qml.qnode(dev)
    def my_circuit(a, b, c):
        """ Args:
                a, b, c (int): The input variables, either 0 or 1.

            Returns:
                array[int]: A single sample of the output of your circuit.
        """

        # YOUR CODE HERE
        # Apply some quantum gates

        return qml.sample()

    return my_circuit


def part_d():
    """Write a quantum circuit that computes the function
        f(a, b) = a + b
    where a, b are two-bit binary values, and + is regular binary addition.

    The first four qubits of your circuit should correspond to the input
    variables, a[0], a[1], b[0], b[1], in that order. The last three qubits of your
    circuit should correspond to the output in the order: carry, (a + b)[0], (a + b)[1].
    If you use an auxiliary wires in your circuit, these should be uncomputed.
           _______
     a0 ---|  C  |--- a0
     a1 ---|  I  |--- a1
     b0 ---|  R  |--- b0
     b1 ---|  C  |--- b1
      0 ---|     |--- 0 # Optional aux wires
             ...
      0 ---|     |--- 0
      0 ---|     |--- carry
      0 ---|     |--- s0
      0 ---=======--- s1

    """

    # You can change this to as many wires as you need
    num_wires = 7

    # Do not change the number of shots on the device
    dev = qml.device("default.qubit", wires=num_wires, shots=1)

    @qml.qnode(dev)
    def my_circuit(a, b):
        """ Args:
                a, b (list([int])): The input variables. For example, a = [1, 0] corresponds
                to a decimal value of 2.

            Returns:
                array[int]: A single sample of the output of your circuit.
        """

        # YOUR CODE HERE
        # Apply some quantum gates

        return qml.sample()

    return my_circuit